In [3]:
import torch.nn as nn
import torchvision.transforms as transforms
import math
import torch
import pdb
import torch.nn as nn
import math
from torch.autograd import Variable
from torch.autograd import Function

import numpy as np


def Binarize(tensor,quant_mode='det'):
    if quant_mode=='det':
        return tensor.sign()
    else:
        return tensor.add_(1).div_(2).add_(torch.rand(tensor.size()).add(-0.5)).clamp_(0,1).round().mul_(2).add_(-1)
#clamp就是超过了range就直接咔嚓掉，截断函数



class HingeLoss(nn.Module):
    def __init__(self):
        super(HingeLoss,self).__init__()
        self.margin=1.0

    def hinge_loss(self,input,target):
            #import pdb; pdb.set_trace()
            output=self.margin-input.mul(target)
            output[output.le(0)]=0 #less or equal!
            return output.mean()

    def forward(self, input, target):
        return self.hinge_loss(input,target)

class SqrtHingeLossFunction(Function):
    def __init__(self):
        super(SqrtHingeLossFunction,self).__init__()
        self.margin=1.0

    def forward(self, input, target):
        output=self.margin-input.mul(target)
        output[output.le(0)]=0
        self.save_for_backward(input, target)
        loss=output.mul(output).sum(0).sum(1).div(target.numel())
        return loss

    def backward(self,grad_output):
       input, target = self.saved_tensors
       output=self.margin-input.mul(target)
       output[output.le(0)]=0
       import pdb; pdb.set_trace()
       grad_output.resize_as_(input).copy_(target).mul_(-2).mul_(output)
       grad_output.mul_(output.ne(0).float())
       grad_output.div_(input.numel())
       return grad_output,grad_output

def Quantize(tensor,quant_mode='det',  params=None, numBits=8):
    tensor.clamp_(-2**(numBits-1),2**(numBits-1))
    if quant_mode=='det':
        tensor=tensor.mul(2**(numBits-1)).round().div(2**(numBits-1))
    else:
        tensor=tensor.mul(2**(numBits-1)).round().add(torch.rand(tensor.size()).add(-0.5)).div(2**(numBits-1))
        quant_fixed(tensor, params)
    return tensor

import torch.nn._functions as tnnf


class BinarizeLinear(nn.Linear):

    def __init__(self, *kargs, **kwargs):
        super(BinarizeLinear, self).__init__(*kargs, **kwargs)

    def forward(self, input):

        if input.size(1) != 784:
            input.data=Binarize(input.data)
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()
        self.weight.data=Binarize(self.weight.org)
        out = nn.functional.linear(input, self.weight)
        if not self.bias is None:
            self.bias.org=self.bias.data.clone()
            out += self.bias.view(1, -1).expand_as(out)

        return out

class BinarizeConv2d(nn.Conv2d):

    def __init__(self, *kargs, **kwargs):
        super(BinarizeConv2d, self).__init__(*kargs, **kwargs)


    def forward(self, input):
        if input.size(1) != 3:
            input.data = Binarize(input.data)
        if not hasattr(self.weight,'org'):
            self.weight.org=self.weight.data.clone()
        self.weight.data=Binarize(self.weight.org)

        out = nn.functional.conv2d(input, self.weight, None, self.stride,
                                   self.padding, self.dilation, self.groups)

        if not self.bias is None:
            self.bias.org=self.bias.data.clone()
            out += self.bias.view(1, -1, 1, 1).expand_as(out)

        return out
import torchvision
import torch.nn.functional as F
import torch.optim as optim
from torch.backends import cudnn
import numpy as np
import os



class MobileNet(nn.Module):
    def __init__(self,num_classes):
        super(MobileNet, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
#                 BinarizeConv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.Hardtanh(inplace=True),
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
#                 BinarizeConv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.Hardtanh(inplace=True),
    
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
#                 BinarizeConv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.Hardtanh(inplace=True),
            )
        def bconv_dw(inp,oup,stride):
            return nn.Sequential(
                BinarizeConv2d(inp,inp,3,stride,1,groups = inp ,bias = True),
                nn.BatchNorm2d(inp),
                nn.Hardtanh(inplace = True),
                BinarizeConv2d(inp,oup,1,1,0,bias = True),
                nn.BatchNorm2d(oup),
                nn.Hardtanh(inplace=True),
            )
        
        self.ratioInfl = 1
        self.model = nn.Sequential(
            conv_bn(  3,  32*self.ratioInfl, 2), 
            conv_dw( 32*self.ratioInfl,  64*self.ratioInfl, 1),
            conv_dw( 64*self.ratioInfl, 128*self.ratioInfl, 2),
            conv_dw(128*self.ratioInfl, 128*self.ratioInfl, 1),
            conv_dw(128*self.ratioInfl, 256*self.ratioInfl, 2),
            bconv_dw(256*self.ratioInfl, 256*self.ratioInfl, 1),
            bconv_dw(256*self.ratioInfl, 512*self.ratioInfl, 2),
            bconv_dw(512*self.ratioInfl, 512*self.ratioInfl, 1),
            conv_dw(512*self.ratioInfl, 512*self.ratioInfl, 1),
            conv_dw(512*self.ratioInfl, 512*self.ratioInfl, 1),
            conv_dw(512*self.ratioInfl, 512*self.ratioInfl, 1),
            conv_dw(512*self.ratioInfl, 512*self.ratioInfl, 1),
            bconv_dw(512*self.ratioInfl, 1024*self.ratioInfl, 2),
            bconv_dw(1024*self.ratioInfl, 1024, 1),
            nn.AvgPool2d(7),
        )
        self.fc = nn.Linear(1024, num_classes)
        self.regime = {
            0: {'optimizer': 'Adam', 'lr': 5e-3},
            25: {'lr': 1e-2},
            50: {'lr': 5e-4},
            75: {'lr': 1e-5},
            100: {'lr': 1e-6},
            150: {'lr': 1e-7},
            200: {'lr': 1e-6},
            250: {'lr': 1e-7},
        }

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

    
def v12_binary(**kwargs):
    num_classes = getattr(kwargs,'num_classes', 10)
    return MobileNet(num_classes)

In [4]:
net = MobileNet(10)
params = list(net.parameters())
k = 0
suma = 0
for i in params:
    l = 1
    print("该层的结构：" + str(list(i.size())))
    for j in i.size():
        l *= j
    print("该层参数和：" + str(l))
    k = k + l
    suma += k
net.parameters

该层的结构：[32, 3, 3, 3]
该层参数和：864
该层的结构：[32]
该层参数和：32
该层的结构：[32]
该层参数和：32
该层的结构：[32, 1, 3, 3]
该层参数和：288
该层的结构：[32]
该层参数和：32
该层的结构：[32]
该层参数和：32
该层的结构：[64, 32, 1, 1]
该层参数和：2048
该层的结构：[64]
该层参数和：64
该层的结构：[64]
该层参数和：64
该层的结构：[64, 1, 3, 3]
该层参数和：576
该层的结构：[64]
该层参数和：64
该层的结构：[64]
该层参数和：64
该层的结构：[128, 64, 1, 1]
该层参数和：8192
该层的结构：[128]
该层参数和：128
该层的结构：[128]
该层参数和：128
该层的结构：[128, 1, 3, 3]
该层参数和：1152
该层的结构：[128]
该层参数和：128
该层的结构：[128]
该层参数和：128
该层的结构：[128, 128, 1, 1]
该层参数和：16384
该层的结构：[128]
该层参数和：128
该层的结构：[128]
该层参数和：128
该层的结构：[128, 1, 3, 3]
该层参数和：1152
该层的结构：[128]
该层参数和：128
该层的结构：[128]
该层参数和：128
该层的结构：[256, 128, 1, 1]
该层参数和：32768
该层的结构：[256]
该层参数和：256
该层的结构：[256]
该层参数和：256
该层的结构：[256, 1, 3, 3]
该层参数和：2304
该层的结构：[256]
该层参数和：256
该层的结构：[256]
该层参数和：256
该层的结构：[256]
该层参数和：256
该层的结构：[256, 256, 1, 1]
该层参数和：65536
该层的结构：[256]
该层参数和：256
该层的结构：[256]
该层参数和：256
该层的结构：[256]
该层参数和：256
该层的结构：[256, 1, 3, 3]
该层参数和：2304
该层的结构：[256]
该层参数和：256
该层的结构：[256]
该层参数和：256
该层的结构：[256]
该层参数和：256
该层的结构：[512, 256, 1, 1]
该层参数和：13107

<bound method Module.parameters of MobileNet(
  (model): Sequential(
    (0): Sequential(
      (0): Conv2d(3, 32, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Hardtanh(min_val=-1.0, max_val=1.0, inplace)
    )
    (1): Sequential(
      (0): Conv2d(32, 32, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), groups=32, bias=False)
      (1): BatchNorm2d(32, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (2): Hardtanh(min_val=-1.0, max_val=1.0, inplace)
      (3): Conv2d(32, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (4): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (5): Hardtanh(min_val=-1.0, max_val=1.0, inplace)
    )
    (2): Sequential(
      (0): Conv2d(64, 64, kernel_size=(3, 3), stride=(2, 2), padding=(1, 1), groups=64, bias=False)
      (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, af

In [6]:
import time
import torch
import torch.nn as nn
import torch.backends.cudnn as cudnn
import torchvision.models as models
from torch.autograd import Variable

class MobileNet(nn.Module):
    def __init__(self):
        super(MobileNet, self).__init__()

        def conv_bn(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, oup, 3, stride, 1, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True)
            )

        def conv_dw(inp, oup, stride):
            return nn.Sequential(
                nn.Conv2d(inp, inp, 3, stride, 1, groups=inp, bias=False),
                nn.BatchNorm2d(inp),
                nn.ReLU(inplace=True),
    
                nn.Conv2d(inp, oup, 1, 1, 0, bias=False),
                nn.BatchNorm2d(oup),
                nn.ReLU(inplace=True),
            )

        self.model = nn.Sequential(
            conv_bn(  3,  32, 2), 
            conv_dw( 32,  64, 1),
            conv_dw( 64, 128, 2),
            conv_dw(128, 128, 1),
            conv_dw(128, 256, 2),
            conv_dw(256, 256, 1),
            conv_dw(256, 512, 2),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 512, 1),
            conv_dw(512, 1024, 2),
            conv_dw(1024, 1024, 1),
            nn.AvgPool2d(7),
        )
        self.fc = nn.Linear(1024, 1000)

    def forward(self, x):
        x = self.model(x)
        x = x.view(-1, 1024)
        x = self.fc(x)
        return x

def speed(model, name):
    t0 = time.time()
    input = torch.rand(1,3,224,224).cuda()
    input = Variable(input, volatile = True)
    t1 = time.time()

    model(input)
    t2 = time.time()

    model(input)
    t3 = time.time()
    
    print('%10s : %f' % (name, t3 - t2))
def speed_cpu(model, name):
    t0 = time.time()
    input = torch.rand(1,3,224,224)
    input = Variable(input, volatile = True)
    t1 = time.time()

    model(input)
    t2 = time.time()

    model(input)
    t3 = time.time()
    
    print('%10s : %f' % (name, t3 - t2))

if __name__ == '__main__':
    #cudnn.benchmark = True # This will make network slow ??
    resnet18 = models.resnet18().cuda()
    alexnet = models.alexnet().cuda()
    vgg16 = models.vgg16().cuda()
    squeezenet = models.squeezenet1_0().cuda()
    mobilenet = MobileNet().cuda()
    net = net.cuda()
    print('inference speed in gpus')
    speed(resnet18, 'resnet18')
    speed(alexnet, 'alexnet')
    speed(vgg16, 'vgg16')
    speed(squeezenet, 'squeezenet')
    speed(mobilenet, 'mobilenet')
    speed(net,'mobilenet_binary')
    
    print('inference speed in cpus')
    resnet18 = resnet18.cpu()
    alexnet = alexnet.cpu()
    vgg16 = vgg16.cpu()
    squeezenet = squeezenet.cpu()
    mobilenet = mobilenet.cpu()
    net = net.cpu()
    speed_cpu(resnet18, 'resnet18')
    speed_cpu(alexnet, 'alexnet')
    speed_cpu(vgg16, 'vgg16')
    speed_cpu(squeezenet, 'squeezenet')
    speed_cpu(mobilenet, 'mobilenet')
    speed_cpu(net,'mobilenet_binary')

inference speed in gpus
  resnet18 : 0.004733
   alexnet : 0.001034
     vgg16 : 0.001882
squeezenet : 0.003498
 mobilenet : 0.006373
mobilenet_binary : 0.006862
inference speed in cpus


/home/data/liuchang/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:58: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.
/home/data/liuchang/anaconda3/envs/py3/lib/python3.7/site-packages/ipykernel_launcher.py:71: UserWarning: volatile was removed and now has no effect. Use `with torch.no_grad():` instead.


  resnet18 : 0.044572
   alexnet : 0.019290
     vgg16 : 0.125436
squeezenet : 0.059603
 mobilenet : 0.036250


RuntimeError: Input type (torch.FloatTensor) and weight type (torch.cuda.FloatTensor) should be the same